# Импорт библиотек

In [19]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

# Подготовка данных

In [20]:
df = pd.read_csv("data/frames_errors.csv", header=None)
df.columns = [
    "block_id",  # ID блока
    "frame_idx",  # ID фрейма в блоке
    "E_mu_Z",  # Факт: Реальный QBER в Z-базисе
    "E_mu_phys_est",  # Альтернативная оценка QBER, основанная на физических параметрах
    "E_mu_X",  # Факт: Реальный QBER в X-базисе
    "E_nu1_X",  # QBER для decoy-состояния nu1 в X-базисе
    "E_nu2_X",  # QBER для decoy-состояния nu2 в X-базисе
    "E_nu1_Z",  # QBER для decoy-состояния nu1 в Z-базисе
    "E_nu2_Z",  # QBER для decoy-состояния nu2 в Z-базисе
    "N_mu_X",  # Количество отправленных сигнальных состояний в X-базисе
    "M_mu_XX",  # Количество принятых сигнальных состояний (отправлено в X, принято в X)
    "M_mu_XZ",  # Количество принятых сигнальных состояний (отправлено в X, принято в Z)
    "M_mu_X",  # Общее количество принятых сигнальных состояний в X-базисе
    "N_mu_Z",  # Количество отправленных сигнальных состояний в Z-базисе
    "M_mu_ZZ",  # Количество принятых сигнальных состояний (отправлено в Z, принято в Z)
    "M_mu_Z",  # Общее количество принятых сигнальных состояний в Z-базисе
    "N_nu1_X",  # Количество отправленных decoy-состояний nu1 в X-базисе
    "M_nu1_XX",
    "M_nu1_XZ",
    "M_nu1_X",
    "N_nu1_Z",
    "M_nu1_ZZ",
    "M_nu1_Z",
    "N_nu2_X",
    "M_nu2_XX",
    "M_nu2_XZ",
    "M_nu2_X",
    "N_nu2_Z",
    "M_nu2_ZZ",
    "M_nu2_Z",
    "nTot",  # Общее количество отправленных состояний
    "bayesImVoltage",  # Напряжение на модуляторе интенсивности
    "opticalPower",  # Оптическая мощность
    "polarizerVoltages[0]",  # Напряжение на контроллере поляризации 0
    "polarizerVoltages[1]",  # Напряжение на контроллере поляризации 1
    "polarizerVoltages[2]",  # Напряжение на контроллере поляризации 2
    "polarizerVoltages[3]",  # Напряжение на контроллере поляризации 3
    "temp_1",  # Температура детектора 1
    "biasVoltage_1",  # Напряжение смещения детектора 1
    "temp_2",  # Температура детектора 2
    "biasVoltage_2",  # Напряжение смещения детектора 2
    "synErr",  # Результат: Ошибка синдрома (1 - коррекция провалилась, 0 - успех)
    "N_EC_rounds",  # Результат: Количество доп. раундов коррекции (индикатор сложности)
    "maintenance_flag",  # Флаг технического обслуживания
    "estimator_name",  # Название алгоритма, который выбрал параметры в реальности
    "f_EC",  # Эффективность коррекции ошибок
    "E_mu_Z_est",  # Предсказание: Оценка QBER, на основе которой были выбраны R, s, p
    "R",  # Параметр: Скорость кода
    "s",  # Параметр: Количество укороченных узлов
    "p",  # Параметр: Количество проколотых узлов
]

df = df.rename(
    columns={
        "block_id": "id",
        "E_mu_Z": "value",
        "frame_idx": "date",
    }
)

In [21]:
missing_counts = df.isna().sum()
columns_with_missing = missing_counts[missing_counts > 0]

if columns_with_missing.empty:
    print("Пропусков в данных нет.")
else:
    print("Столбцы с пропусками и их количество:")
    print(columns_with_missing)

Столбцы с пропусками и их количество:
E_mu_phys_est    448
f_EC             131
dtype: int64


In [22]:
df_for_ts = df.set_index(["id", "date"]).unstack().ffill().stack().reset_index()
timestamp_counts = df_for_ts.groupby("id")["date"].nunique()
print("Количество фреймов/Количество рядов")
print(timestamp_counts.value_counts())

Количество фреймов/Количество рядов
date
400    815
399      8
Name: count, dtype: int64


In [14]:
df_base = df.drop(
    [
        "E_mu_phys_est",
        "f_EC",
    ],
    axis=1,
)

In [23]:
df = df_base.copy()

In [27]:
timestamp_counts = df.groupby("id")["date"].nunique()
print("Количество фреймов / Количество рядов")
print(timestamp_counts.value_counts())

Количество фреймов / Количество рядов
date
399    569
400    251
398      2
390      1
Name: count, dtype: int64


In [ ]:
df_for_ts = df[["id", "value", "date"]].dropna(subset=["value"], how="any")

In [10]:
df_for_ts = df_for_ts.set_index(["id", "date"]).unstack().ffill().stack().reset_index()
timestamp_counts = df_for_ts.groupby("id")["date"].nunique()
print("Количество фреймов/Количество рядов")
print(timestamp_counts.value_counts())

Количество фреймов/Количество рядов
date
400    815
399      8
Name: count, dtype: int64


/tmp/ipykernel_5569/143935003.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_for_ts = df_for_ts.set_index(["id", "date"]).unstack().ffill().stack().reset_index()


In [11]:
df_for_ts = df_for_ts.groupby("id").filter(lambda x: len(x) == 400)
print("Оставшиеся сегменты:", df_for_ts["id"].nunique())

Оставшиеся сегменты: 815


In [20]:
start_idx = df_for_ts[(df_for_ts["id"] == 1489460492) & (df_for_ts["date"] == 99)].index[0]
end_idx = df_for_ts[(df_for_ts["id"] == 1840064900) & (df_for_ts["date"] == 101)].index[0]

df_submit = df_for_ts.loc[start_idx:end_idx].copy()

In [21]:
df_submit

,id,date,value
224091,1489460492,99,0.02507
224092,1489460492,100,0.02121
224093,1489460492,101,0.02029
224094,1489460492,102,0.01904
224095,1489460492,103,0.02018
...,...,...,...
276489,1840064900,97,0.01890
276490,1840064900,98,0.01897
276491,1840064900,99,0.01838
276492,1840064900,100,0.02000


In [17]:
len(df_for_ts["id"].unique())

815